### Entrega Gross-Pitaevskii 2025
##### Lourdes Simón y Ana Matas

**Función de onda y su expresión:**

La descripción teórica de la condensación de Bose-Einstein se basa en la ecuación de Gross-Pitaevskii (G-P). Los átomos confinados por un campo magnético se describen muy bien mediante un potencial de oscilador armónico.
En unidades de oscilador la ecuación se escribe

$$ \Big[-\frac{1}{2} \nabla^2 + \frac{1}{2}r^2 + 4 \pi a_s N |\psi|^2 \Big]\psi = \mu \psi \tag{1}$$

donde $\mu$ es el potencial químico, N el numero de átomos y $a_s$ la longitud de dispersión (también en unidades  del oscilador armónico).

Consideramos la función de onda
$$\psi(\vec{r}) = \frac{R(r)}{r}\,Y_{00}(\theta,\phi)$$

donde el armónico esférico es $Y_{00}(\theta,\phi) = \frac{1}{\sqrt{4\pi}}.$

De esta manera, la ecuación (G-P) puede escribirse como

$$ -\frac{1}{2} \frac{d^2R(r)}{dr^2} + \frac{1}{2}r^2R(r) + a_s N\frac{|R(r)|^2}{r^2}R(r) = \mu R(r)$$

Imponiendo la condición de normalización, considerando que $Y_{00}$ está normalizado y suponiendo qur R(r) tiene la siguiente estructura: $R(r) = Cre^{-\frac{1}{2}\alpha^2r^2}$, se procederá calculando su constante de normalización C:

$$\int |\psi(\vec{r})|^2 \ dr^3 = \int_{0}^{\infty} \left|\frac{R(r)}{r}\right|^2 r^2\ dr \ \int |Y_{00}(\theta,\phi)|^2\ d\Omega = \int_{0}^{\infty} |R(r)|^2 \ dr = C^2\int_{0}^{R} r^2 e^{-\alpha^2r^2} dr = C^2 \frac{\sqrt{\pi}}{4\alpha^3}=   1.$$

De esta manera se encuentra el siguiente valor para la constante: $ C=\frac{2\alpha^{3/2}}{\pi^{1/4}} $. Además se impondrá $R(0)=0$.

En el código que presentamos a continuación utilizamos un método iterativo para ir acercándonos al estado estacionario de la función de onda solución de G-P. Para ello, partimos de la función de onda radial del estado fundamental (G.S.) del oscilador armónico 3D

$$R_0(r) = \frac{2\ \alpha^{3/2}}{\pi^{1/4}}\ r\ \exp\Bigl(-\frac{1}{2}\alpha^2r^2\Bigr)$$



In [1]:
# Entrega Gross-Pitaevskii 2025
# Lourdes Simón y Ana Matas 

# En este código se resuelve la ecuación de Gross-Pitaevskii de manera iterativa y se estudia 
# la evolución de valores como la energía, la densidad, el potencial químico...

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import imageio.v2 as imageio


a0 = 0.00433 # scattering length en unidades del oscilador.
n1 = 900 # número de puntos en la discretizacion de r.
step = 0.015 # equiespaciado de los puntos en el espacio de r.
atoms = 1000000 # número de átomos.
time_step = 0.00005 # equiespaciado de los puntos en el espacio de tiempo.
alpha = 0.3 # parámetro inicial para la función de onda G.S. del oscilador armónico.
niter = 15000 # número de iteraciones.

# Se definen las constantes
pi = np.pi
const = 2.0*(alpha**(3/2))/(pi**(1/4)) # constante de normalización de la función inicial 

# array con n1 valores equiespaciados (step) de r
xr = np.array([step*float(i) for i in range(n1)], dtype=np.float64)

# Se define la función de onda inicial R(r) = const*r*exp(-0.5*(alpha^2)*r^2)
f_ini = np.zeros_like(xr) # función de onda inicial
f_old = np.zeros_like(xr) # copia que se usará para calcular las derivadas

xr2 = xr**2
f_ini = const*xr*np.exp(-0.5*(alpha**2)*xr2) # empezamos con la función de onda G.S. del oscilador
f_old = f_ini.copy()

In [ ]:
plt.plot(xr, f_ini, label=r'$R_0(r)$')
plt.xlabel('r')
plt.ylabel(r'$R_0(r)$')
plt.title('Función de onda radial inicial (oscilador armónico)')
plt.grid(True)
plt.legend()
plt.show()

Continuaremos con la evolución temporal del programa. La ecuación de Schrödinger dependiente del tiempo se expresa como:

$$ i\hbar \frac{\partial \psi}{\partial t} = H\psi$$


En este programa se pretende utilizar una evolución temporal con tiempo imaginario, donde se realiza el siguiente cambio de variables: $t \rightarrow -i\tau $. De esta manera la ecuación G-P se transforma en una ecuación de difusión, cuyo resultado es el estado estacionario del sistema. 

$$ \frac{\partial \psi}{\partial \tau} = -\frac{1}{\hbar}H\psi$$

Con solución: $\psi (\tau)=\sum_n c_ne^{-E_n\tau / \hbar}\phi_n$, donde $\phi_n$ son los autovectores de energía $E_n$ ordenados segun $E_0 < E_1 < ...$. De esta manera, si la función de onda de partida es una combinación lineal de estados que incluye el fundamental, este decae más lentamente mientras que los estados excitados lo hacen más rápido. La evolución tiende a eliminar las contribuciones de los estados excitados, haciendo que la función de onda converja al estado estacionario de mínima energía.

La función de onda tras una iteración, considerando $\Delta \tau$ pequeño, será: $ \psi (x, \tau + \Delta \tau) = \psi (x, \tau) - \frac{\Delta \tau H }{\hbar} \psi (x, \tau)$, donde $\hbar =1$ y $H \psi=\mu \psi$.

----------------------

**Cálulos realizados en el programa:**

Para calcular el laplaciano utilizamos la siguiente aproximación numérica del cálculo de la segunda derivada:
$$\frac{d^2f}{dx^2}=\frac{\frac{f(x+dx)-f(x)}{dx} - \frac{f(x)-f(x-dx)}{dx}}{dx}.$$

Además, será necesario ir normalizando las funciones de onda ya que con este proceso iterativo pierden norma. Así pues, se garantiza que corresponda al valor fundamental que se pretende encontrar.

Para calcular la energía por partícula partimos de la expresión:
$$ \frac{E}{N} = \int dr^3 \psi^* [-\frac{1}{2} \nabla^2 + \frac{1}{2}r^2]\psi + 2 \pi a_s N \int dr^3 |\psi|^4 $$

que en nuestro caso, considerando la función de onda utilizada, se puede escribir como:
$$ \frac{E}{N} = \int dr \Bigr(-\frac{1}{2}R  \frac{d^2R}{dr^2} + \frac{1}{2}r^2 R^2 \Bigl) + \frac{1}{2} a_s N \int dr \Bigr|\frac{R}{r}\Bigl|^4.$$

Respecto al método iterativo, en lugar de un número muy alto de iteraciones se puede poner una condición de convergencia. En nuestro código hay un apartado comentado (ya que finalmente no se ha utilizado) sobre una posible condición que corresponde a mirar el "overlap" de las funciones de onda antes y después de una iteración. Cuando este se acerca mucho a 1 se considera que ha convergido. No hemos podido encontrar un threshold adecuado, ya que si es muy pequeño se hacen todas las iteraciones pero si es muy grande solo se hace una. Por eso hemos realizado los cálculos con un número de iteraciones alto (50000) para asegurar la convergencia, sin usar esta condición.

In [ ]:
cequ = a0*atoms # parametro de la interacción (no linealidad), si cequ = 0 se recupera el oscilador armónico puro
# cequ = 0.0

# Se crean los arrays necesarios para las iteraciones, inicialmente con ceros.
f_der = np.zeros_like(xr) # segunda derivada
f_new = np.zeros_like(xr) # función de onda nueva actualizada
mu = np.zeros_like(xr) # potencial químico

itw = 0  # contador para imprimir cada 200 iteraciones (solo para controlar la evolución).

# Creamos una carpeta para guardar las imágenes de la función de onda radial R(r) en cada iteración. Posteriormente se
# usará para ver la evolución.
carpeta = "plots_R(r)"
os.makedirs(carpeta, exist_ok=True)

# Bucle iterativo para convergencia.
for it in range(1, niter+1):
    itw += 1
    norm = 0.0 # norma de la función de onda (se tiene que ir normalizando ya que con el tiempo pierde norma)
    ene0  = 0.0
    
    # Calculo del laplaciano de la función de onda (término cinético).
    f_der[1:-1] = (f_old[:-2] + f_old[2:] - 2.0*f_old[1:-1])/(step**2)
    f_der[-1] = (f_old[-2] - 2.0 * f_old[-1])/(step**2) # el punto de la frontera lo calculo aparte
    # dado que imponemos que R(0)=0, mantenemos f_der[0]=0.0

    # Actualización de la función de onda, cálculo de energía y potencial químico. Dentro de la energía se tienen que calcular las
    # contribuciones de la energía cinética, potencial y de interacción:
    for i in range(n1):
        xr2 = xr[i] * xr[i]
        if i == 0:
            mu[i] = 0.0 # evitamos dividir por 0
        else:
            ene0 += (-0.5*f_old[i]*f_der[i] + 0.5*xr2*(f_old[i]**2) + 0.5*cequ*xr2*((f_old[i]/xr[i])**4)) # energía por partícula
            mu[i] = (-0.5*f_der[i]/f_old[i] + 0.5*xr2 + cequ*((f_old[i]/xr[i])**2))
            # Para la aproximación Thomas-Fermi:
            # ene0 += ( 0.5*xr2*(f_old[i]**2) + 0.5*cequ*xr2*((f_old[i]/xr[i])**4))
            # mu[i] = ( 0.5*xr2 + cequ*((f_old[i]/xr[i])**2)) # (aprox T-F)

        # Evolución de la función de onda tras una iteración
        f_new[i] = f_old[i] - time_step*mu[i]*f_old[i]
        norm += f_new[i]**2
    
    # Calculo de la norma como la raíz de la integral (la integral se aproxima como la suma de los puntos por el step)
    norm = math.sqrt(norm*step)
    ene0  = ene0*step
    
    # Normalizamos la nueva función de onda
    f_new = f_new/norm

    
    # # Calculo del overlap entre la iteración anterior y esta
    # overlap = np.sum(f_new*f_old)*step

    # # Si el overlap se acerca a 1 consideramos que ha convergido
    # if abs(1 - overlap) < 0.000005:
    #     print("Se ha alcanzado la convergencia en la iteración", it, "con overlap =", overlap)
    #     break
    
    
    # Salida periódica cada 200 iteraciones.
    if itw == 200:
        print("Iteración %d, ene0 = %12.5e" % (it, ene0))
        itw = 0
        
    # Guardamos la imagen de R(r) cada 10 iteraciones para poder graficar luego la evolución en GIF
    if it % 10 == 0:
        fig, ax = plt.subplots()
        ax.plot(xr, f_new, label=f"Iteración {it} - Energía={ene0:.5f}")
        ax.set_xlabel("r")
        ax.set_ylabel("R(r)")
        ax.set_ylim(0, 0.6)
        ax.set_title(f"Función de onda radial con N=1.000.000 átomos")
        ax.grid()
        ax.legend()
        fig.tight_layout()  # ajustamos los márgenes de la figura para que no se solapen las etiquetas/leyendas
        fig.savefig(f"{carpeta}/R_{it:04d}.png", facecolor="white") # guardamos el plot en una carpeta previamente creada
        plt.close(fig)
    
    # Asignamos a la variable f_old la nueva función.
    f_old = f_new
    

In [4]:
# Creación del gif para observar la evolución y poder determinar cuándo ha convergido

# Filtramos los archivos que son imágenes ".png" y mediante sorted(...,key=...) ordenamos las imágenes según el número de iteración 
imagenes = sorted(
    [img for img in os.listdir(carpeta) if img.endswith(".png")],
    key=lambda x: int(x.split("_")[1].split(".")[0])  # mediante este comando se convierte el nombre del archivo a entero para una correcta ordenación
)

# Leemos la imagen como un vector de píxeles y los junta en la lista. Cada frame es una de las gráficas anteriores bien ordenadas
frames = [imageio.imread(os.path.join(carpeta, img)) for img in imagenes]

# Guardamos y mostramos todos frames en un solo GIF:
imageio.mimsave("evolucion_R(r).gif", frames, fps=10)

In [ ]:
# Grafica de la función de onda final y potencial químico
plt.figure()
plt.plot(xr, f_old, label=r"$R(r)$ final")
plt.xlabel("r")
plt.ylabel(r"$R(r)$")
plt.grid()
plt.title("Función de onda radial final")
plt.legend()

plt.figure()
plt.plot(xr[1:], mu[1:], label=r"$\mu(r)$")
plt.xlabel("r")
plt.ylabel(r"$\mu(r)$")
plt.grid()
plt.title("Potencial químico final")
plt.legend()

plt.show()

In [6]:
# Recalculamos el laplaciano con la última funcion obtenida para estudiar valores esperados.
f_der[1:-1] = (f_old[:-2] + f_old[2:] - 2.0*f_old[1:-1])/(step**2)
f_der[-1] = (f_old[-2] - 2.0 * f_old[-1])/(step**2)

# Variables para integrales finales.
radio = 0.0 # radio cuadrático medio
ekin = 0.0 # energía cinética
pot_ho = 0.0 # energía potencial del oscilador armónico
pot_int = 0.0 # energía de interacción
mu_1 = 0.0 # potencial químico
xaver = 0.0 # densidad media ponderada con la densidad (proporcional al número de partículas)
normden = 0.0 # normalizació de la densidad

# Se crean arrays para densidad y potencial unipersonal
den = np.zeros_like(xr) # densidad total
u = np.zeros_like(xr) # potencial total: V(r)+g

# Calcular valores esperados
for i in range(1, n1):
    xr2 = xr[i]*xr[i]
    radio += xr2*(f_old[i]**2)
    ekin += -0.5*f_old[i]*f_der[i]
    pot_ho += 0.5*xr2*(f_old[i]**2)
    pot_int += 0.5*cequ*xr2*((f_old[i]/xr[i])**4)
    mu_1 += mu[i]*(f_old[i]**2)
    u[i] = 0.5*xr2 + cequ*((f_old[i]/xr[i])**2)
    den[i] = (f_old[i] / xr[i])**2
    normden += den[i]*xr2

# Integrar (multiplicar por el paso 'step')
radio = math.sqrt(radio*step)
mu_1 = mu_1*step
ekin = ekin*step
# ekin = 0  # en la aproximación T-F
pot_ho = pot_ho*step
pot_int = pot_int*step
pot = pot_int + pot_ho
normden = normden*step

In [ ]:
# Utilizado para la comparación de la densidad sin y con la aproximación T-F para dos valores de N.
#den_1000 = den.copy()
#den_tf_1000 = den.copy()
#den_100000 = den.copy()
#den_tf_100000 = den.copy()

In [8]:
# Plots de la comparación de densidades con y sin aproximación T-F

# plt.figure()
# plt.plot(xr[1:], den_1000[1:], color='red', label=r"Gross-Pitaevskii")
# plt.plot(xr[1:], den_tf_1000[1:], color='blue', label=r"Thomas-Fermi")
# plt.xlabel("r")
# plt.ylabel("$|\psi (r)|^2$")
# plt.title("Comparación de Densidad N=1000")
# plt.grid()
# plt.legend()
# plt.savefig("densidad_comparada_N1000.png", format="png", dpi=300)
# plt.show()

# plt.figure()
# plt.plot(xr[1:], den_100000[1:], color='red', label=r"Gross-Pitaevskii")
# plt.plot(xr[1:], den_tf_100000[1:], color='blue', label=r"Thomas-Fermi")
# plt.xlabel("r")
# plt.ylabel("$|\psi (r)|^2$")
# plt.title("Comparación de Densidad N=100000")
# plt.grid()
# plt.legend()
# plt.savefig("densidad_comparada_N100000.png", format="png", dpi=300)
# plt.show()

In [ ]:
# Tabla de resultados
virial = 2.0*ekin - 2.0*pot_ho + 3.0*pot_int
mu_analitico = (15.0*a0*atoms)**(2.0/5.0)/2.0
diferencia_mu = abs(mu_1-mu_analitico)

results = pd.DataFrame({
    "Observable": ["E", "E_kin", "E_pot_ho", "E_pot_int", "E_pot_total", "μ", 
        "Norma densidad", "Teorema_Virial", "μ_analitico", "dif_mu" , "radio"],
    "Valor": [ene0, ekin, pot_ho, pot_int, pot, mu_1, normden, virial, mu_analitico, diferencia_mu, radio]})

print("\n--- Observables finales ---")
print(results.to_string(index=False, float_format="{:15.10f}".format, justify='center'))

# Imprimir resultados finales

-----------------------------------------------------------
## **Resultados**

#### **Sin interacción**
**a0)** Si la interacción es nula (cequ=0), deberíamos obtener la función de onda del oscilador armónico. La energía por partícula del G.S. en unidades del oscilador debería ser 3/2 (estamos en 3D). Además, si no tenemos interacción, la ecuación (1) se reduce a

$$ \Big[-\frac{1}{2} \nabla^2 + \frac{1}{2}r^2 \Big] \psi = \mu \psi$$

Por tanto, el potencial $\mu$ es el autovalor de la energía por partícula y también debería dar 3/2.

Para 50000 iteraciones obtenemos
- Energia por particula = 1.50001552e+00  
- Potencial quimico = 1.50001551e+00

Tambien vemos que se cumple, con una precisión muy alta, el teorema del Virial $V=E_{kin}$:
- $E_{kin} - V$ = -0.024588

#### **Dependencia con N**
**a)** Podemos estudiar la dependencia de varios valores con el numero de átomos N. A continuación presentamos una tabla con los resultados. Recordamos que todo està en unidades del oscilador.


|   N    |     $\mu$      |      $e$      |    $e_{kin}$    |    $e_{ho}$     |    $e_{int}$    | Teorema Virial |
|:------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|:-------------:|
|  100   | 1,78642630  | 1,65197295  | 0,652650434 | 0,864869027 | 0,134453482 | -0,0210767406 |
|  1000  | 3,04385962  | 2,42467164  | 0,437058801 | 1,36842477  | 0,619188069 | -0,0051677411 |
| 10000  | 6,86574966  | 5,04153096  | 0,240332013 | 2,97698023  | 1,82421872  | -0,0006402840 |
|100000  | 16,8465867  | 12,1039499  | 0,123670621 | 7,23764249  | 4,74263679  | -0,0000333726 |
|1000000 | 42,1191624  | 30,1200986  | 0,0612203585| 18,0598146  | 11,9990637  | 0,0000027403  |


En esta tabla podemos ver para 5 valores distintos de N el potencial químico $\mu$, las energías por partícula total $e$, cinética $e_{kin}$, del oscilador armónico $e_{ho}$ y de interacción $e_{int}$. También podemos ver que se cumple (de manera aproximada) el teorema de Virial: $2 e_{kin} - 2 e_{ho} + 3 e_{int}=0$.

Observamos que todas las magnitudes aumentan con el numero de átomos N, exceptuando la energía cinética. Es un resultado esperable ya el valor del término de interacción se hace más significativo a medida que N aumenta, por lo que la contribución de la energía de interacción crece con N. Cuando el término de interacción se hace más fuerte, el condensado se expande. La función de onda se vuelve más ancha y las variaciones más suaves, por lo que el término cinético (relacionado con la segunda derivada) se reduce. 

Por otra parte, se puede interpretar $\mu$ como la energía necesaria para quitar o añadir una partícula del condensado, es decir, $\frac{dE}{dN}$. La energía total aumenta con N y cada vez más rápidamente, ya que si N es muy grande, al introducir una nueva partícula en el sistema incrementa el número de nuevas parejas que contribuyen a la energía de interacción y en consecuencia a la energía total $E$. De este modo, con N mas grandes se requiere una mayor energía para añadir o extraer una partícula y $\mu$ aumenta. 

Finalmente, cabe destacar que el teorema de Virial se cumple con mayor exactitud para valores altos de N.


#### **Aproximacion Thomas-Fermi**

La aproximación de Thomas-Fermi consiste en no considerar la contribución de la energía cinética. Esta aproximación es caorrecta cuando la interacción entre partículas es muy intensa, por lo que se puede menospreciar la parte cinética.

**b)** Se repetirá el mismo estudio aplicando la aproximación Thomas-Fermi. En este caso la ecuación G-P se reduce a 

$$ \frac{1}{2}r^2R(r) + a_s N\frac{|R(r)|^2}{r^2}R(r) = \mu R(r)$$

Obtenemos los siguientes resultados:

|   N    |     $\mu$      |      $e$      |    $e_{kin}$    |    $e_{ho}$     |    $e_{int}$    | Teorema Virial |
|:------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|:---------:|
| 100     | 1,03088767  | 0,777570515 | 0,00000000   | 0,524249659  | 0,253319741  | -0,2885400946  |
| 1000    | 2,63208215  | 1,90141300  | 0,00000000   | 1,17074244   | 0,73067029   | -0,1494740086  |
| 10000   | 6,65402918  | 4,76404995  | 0,00000000   | 2,87407011   | 1,88997978   | -0,0782008628  |
| 100000  | 16,7422848  | 11,9641363  | 0,0000000    | 7,18598759   | 4,77814871   | -0,0375290355  |
| 1000000 | 42,0704923  | 30,0520433  | 0,0000000    | 18,0335943   | 12,0184490   | -0,0118415340  |

El comportamiento de las magnitudes estudiadas es el mismo si realizamos esta aproximación. 

Respecto a los valores obtenidos, vemos que las discrepancias con el caso anterior (sin aproximación) son mayores para valores bajos de N. Esto se debe ya que a medida que N aumenta, la interacción se vuelve más importante y el termino cinético más pequeño, y en consecuencia, la aproximación realizada es más acertada y se garantiza una mayor exactitud. Los resultados concuerdan con lo expuesto al inicio.

Por otra parte también se determina que para esta aproximación, el teorema de Virial no se cumple con tanta exactitud, pero igualmente mejora a medida que N aumenta.

#### **Perfil de densidad**
**c)** A continuación presentamos la representación gráfica de la densidad $\rho (r) = |\psi|^2$ para N=1000 y N=100000. Se compara el resultado con la aproximación T-F.

<div style="display: flex; justify-content: center; align-items: center;">
  <div style="margin: 10px;">
    <img src="densidad_comparada_N1000.png" alt="Imagen 1" style="max-width: 100%;">
  </div>
  <div style="margin: 10px;">
    <img src="densidad_comparada_N100000.png" alt="Imagen 2" style="max-width: 100%;">
  </div>
</div>

Como bien se ha argumentado en los apartados a) y b), para un mayor número de átomos en el condensado, la aproximación de TF se asemeja más al resultado esperado ya que la contribución de la energía cinética es menor e incluso negligible. Este echo queda reflejado en las gráficas obtenidas.



-------------------------------------------------

#### Apartados extra

**Solución analítica de la aproximación Thomas-Fermi:**

En uno de los apartados del problema se ha estudiado la evolución mediante la aproximación de Thomas-Fermi y se ha establecido  su validez. Si bien se ha resuelto el problema mediante la evolución de la función de onda recurriendo al sistema del tiempo imaginario, en esta apartado se pretende comparar el resultado obtenido con la resolución  analítica.

Recordemos la ecuación en unidades de oscilador, véase expresión (1). Anulando el término cinético y aislando la densidad de la función de onda se obtiene:

$$ \rho(r) = |\psi|^2 = \frac{\mu - \frac{1}{2}r^2}{4 \pi a_s N} $$

Normalizando la densidad entre los márgenes permitidos, es decir, entre $r=0$ y $r=\sqrt{2\mu}$, donde la función de onda puede existir ya que su densidad es positiva, se puede encontrara el valor de $\mu$:

$$ 1 = \int_{0}^{\sqrt{2\mu}} \rho(r) d^3r = \frac{1}{4\pi a_s N} \int_{0}^{\sqrt{2\mu}}(\mu - \frac{1}{2}r^2) 4\pi r^2 dr = \frac{1}{a_s N}(\frac{\mu r^3}{3} - \frac{r^5}{10})\Big|_{0}^{\sqrt{2\mu}} = \frac{(2\mu)^{5/2}}{15 a_s N} $$


$$\mu_{analítico} = \frac{(15 a_s N)^{2/5}}{2}$$

Sustituyendo valores y comparando con el préviamente calculado se observa:

|   N    |     $\mu$      |      $\mu_{analitico}$      |    $\Delta \mu$ |
|:------:|:-----------:|:-----------:|:-----------:|
| 100     | 1,03088767  | 1,0568215645 | 0,0259338898   | 
| 1000    | 2,63208215  | 2,6546157484  | 0,0225335975   | 
| 10000   | 6,65402918  | 6,6680932793  | 0,0140641010   | 
| 100000  | 16,7422848  | 16,7494930323  | 0,0072082689    | 
| 1000000 | 42,0704923  | 42,0728242824  | 0,0023319741    | 

La diferencia entre valores es practicamente nula y decrece a medida que aumenta el número de átomos del condensado. Así pues, la diferencia entre ambos métodos reside en los efectos de contorno, donde la densidad  $\rho(r)$ en el método analítico se anula bruscamente cuando $V(r)=\mu$. Al resolverlo mediante el método iterativo se consigue un perfil más suavizado cerca del borde del condensado. Al aumentar el número de átomos, estos efectos son menos relevantes respecto al volumen total del condensado, disminuyendo las diferencias entre ambas soluciones.

**Evolución de la función de onda radial $R(r)$:**

Finalmente, hemos representado la evolución de la parte radial de la función de onda $R(r)$ a lo largo de las iteraciones para N=1000000. Se ha indicado también la energía calculada en cada una de ellas para poder ver a partir de qué iteración se puede considerar que ha convergido. Al principio evoluciona rápidamente y a medida que avanzan las iteraciones va disminuyendo la velocidad y los cambios de $R(R)$ son más sutiles. Para este apartado hemos reducido las iteraciones a 15000, a partir de entonces vemos que la energía ya no disminuye.

Se puede ver el GIF una vez finalizado fuera del codigo. Lo añadiremos a la carpeta zip ya que no podemos mostrarlo aqui. 